In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import sys

In [11]:
tf.__version__

'2.3.0'

# tf1 record example

In [2]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

## write tfrecord file

In [6]:
## data
corpus = np.array([[[1,2],[0,1]],[[2,4],[5,6]],[[4,2],[0,0]]])
seq_len = np.array([2,2,1], dtype=np.int64)

## write tfrecord file 
filename = 'test2.tfrecords'  # address to save the TFRecords file
# tf1 的老版本问题
# writer = tf.python_io.TFRecordWriter(filename) # AttributeError: module 'tensorflow' has no attribute 'python_io'
writer = tf.compat.v1.python_io.TFRecordWriter(filename)

In [7]:
for i in range(len(corpus)):    
    feature = {"corpus" : _bytes_feature(tf.compat.as_bytes(corpus[i].tostring())),
               "seq_len" : _int64_feature(seq_len[i])}
    example = tf.train.Example(features=tf.train.Features(feature=feature))
    writer.write(example.SerializeToString())

In [8]:
writer.close()
sys.stdout.flush()

## read tfrecord file

In [24]:
## read tfrecord file 
corpus_list = []
seq_list = []

# with tf.Session() as sess:
with tf.compat.v1.Session() as sess:
    feature = {'corpus': tf.io.FixedLenFeature([], tf.string),  #tf.FixedLenFeature -> tf.io.FixedLenFeature
               'seq_len': tf.io.FixedLenFeature([], tf.int64)}
    # Create a list of filenames and pass it to a queue
    filename_queue = tf.compat.v1.train.string_input_producer(['test.tfrecords'], num_epochs=1)
    # Define a reader and read the next record
    # reader = tf.TFRecordReader()
    reader =  tf.compat.v1.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    # Decode the record read by the reader
    features = tf.compat.v1.parse_single_example(serialized_example, features=feature)
    # Convert the image data from string back to the numbers
    corpus = tf.compat.v1.decode_raw(features['corpus'], tf.int32)   
    # Cast label data into int32
    seq_len = tf.cast(features['seq_len'], tf.int32)
    corpus = tf.reshape(corpus, [2,2])
    # Creates batches by randomly shuffling tensors
    corpus_, seq_len_ = tf.compat.v1.train.shuffle_batch([corpus, seq_len], batch_size=1, capacity=30, num_threads=1, min_after_dequeue=10)
    init = tf.group(tf.compat.v1.global_variables_initializer(), tf.compat.v1.local_variables_initializer())
    sess.run(init)
    # Create a coordinator and run all QueueRunner objects
    coord = tf.train.Coordinator()
    threads = tf.compat.v1.train.start_queue_runners(coord=coord)
    for i in range(2):
        test_corpus,test_seq_len = sess.run([corpus_, seq_len_])
        corpus_list.append(test_corpus)
        seq_list.append(test_seq_len)

INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.NotFoundError'>, 2 root error(s) found.
  (0) Not found: test.tfrecords; No such file or directory
	 [[{{node ReaderReadV2_3}}]]
	 [[ParseSingleExample_2/ParseExample/ParseExampleV2/_65]]
  (1) Not found: test.tfrecords; No such file or directory
	 [[{{node ReaderReadV2_3}}]]
0 successful operations.
0 derived errors ignored.


OutOfRangeError: ignored

# tf2 record example
https://www.tensorflow.org/tutorials/load_data/tfrecord

为了高效地读取数据，比较有帮助的一种做法是对数据进行序列化并将其存储在一组可线性读取的文件（每个文件 100-200MB）中。这尤其适用于通过网络进行流式传输的数据。这种做法对缓冲任何数据预处理也十分有用。

TFRecord 格式是一种用于存储二进制记录序列的简单格式。

协议缓冲区是一个跨平台、跨语言的库，用于高效地序列化结构化数据。

协议消息由 .proto 文件定义，这通常是了解消息类型最简单的方法。

tf.Example 消息（或 protobuf）是一种灵活的消息类型，表示 {"string": value} 映射。它专为 TensorFlow 而设计，并被用于 TFX 等高级 API。

本笔记本将演示如何创建、解析和使用 tf.Example 消息，以及如何在 .tfrecord 文件之间对 tf.Example 消息进行序列化、写入和读取。

注：这些结构虽然有用，但并不是强制的。您无需转换现有代码即可使用 TFRecord，除非您正在使用 tf.data 且读取数据仍是训练的瓶颈。有关数据集性能的提示，请参阅数据输入流水线性能。

## 设置

In [25]:
import tensorflow as tf

import numpy as np
import IPython.display as display

In [26]:
tf.__version__

'2.3.0'

## tf.Example
### tf.Example 的数据类型
从根本上讲，tf.Example 是 {"string": tf.train.Feature} 映射。

tf.train.Feature 消息类型可以接受以下三种类型（请参阅 .proto 文件）。大多数其他通用类型也可以强制转换成下面的其中一种：

* tf.train.BytesList（可强制转换自以下类型）
 * string
 * byte
* tf.train.FloatList（可强制转换自以下类型）
 * float (float32)
 * double (float64)
* tf.train.Int64List（可强制转换自以下类型）
 * bool
 * enum
 * int32
 * uint32
 * int64
 * uint64

为了将标准 TensorFlow 类型转换为兼容 tf.Example 的 tf.train.Feature，可以使用下面的快捷函数。请注意，每个函数会接受标量输入值并返回包含上述三种 list 类型之一的 tf.train.Feature：

In [27]:
# The following functions can be used to convert a value to a type compatible
# with tf.Example.

def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

注：为了简单起见，本示例仅使用标量输入。要处理非标量特征，最简单的方法是使用 tf.io.serialize_tensor 将张量转换为二进制字符串。在 TensorFlow 中，字符串是标量。使用 tf.io.parse_tensor 可将二进制字符串转换回张量。

下面是有关这些函数如何工作的一些示例。请注意不同的输入类型和标准化的输出类型。如果函数的输入类型与上述可强制转换的类型均不匹配，则该函数将引发异常（例如，_int64_feature(1.0) 将出错，因为 1.0 是浮点数，应该用于 _float_feature 函数）：

In [28]:
print(_bytes_feature(b'test_string'))
print(_bytes_feature(u'test_bytes'.encode('utf-8')))

print(_float_feature(np.exp(1)))

print(_int64_feature(True))
print(_int64_feature(1))

bytes_list {
  value: "test_string"
}

bytes_list {
  value: "test_bytes"
}

float_list {
  value: 2.7182817459106445
}

int64_list {
  value: 1
}

int64_list {
  value: 1
}



In [29]:
# 可以使用 .SerializeToString 方法将所有协议消息序列化为二进制字符串：
feature = _float_feature(np.exp(1))
feature.SerializeToString()

b'\x12\x06\n\x04T\xf8-@'

## 创建 tf.Example 消息

假设您要根据现有数据创建 tf.Example 消息。在实践中，数据集可能来自任何地方，但是从单个观测值创建 tf.Example 消息的过程相同：   

*  在每个观测结果中，需要使用上述其中一种函数，将每个值转换为包含三种兼容类型之一的 tf.train.Feature。

* 创建一个从特征名称字符串到第 1 步中生成的编码特征值的映射（字典）。

* 将第 2 步中生成的映射转换为 Features 消息。    

在此笔记本中，您将使用 NumPy 创建一个数据集。

此数据集将具有 4 个特征：   

* 具有相等 False 或 True 概率的布尔特征
* 从 [0, 5] 均匀随机选择的整数特征
* 通过将整数特征作为索引从字符串表生成的字符串特征
* 来自标准正态分布的浮点特征      
请思考一个样本，其中包含来自上述每个分布的 10,000 个独立且分布相同的观测值：

In [30]:
# The number of observations in the dataset.
n_observations = int(1e4)

# Boolean feature, encoded as False or True.
feature0 = np.random.choice([False, True], n_observations)

# Integer feature, random from 0 to 4.
feature1 = np.random.randint(0, 5, n_observations)

# String feature
strings = np.array([b'cat', b'dog', b'chicken', b'horse', b'goat'])
feature2 = strings[feature1]

# Float feature, from a standard normal distribution
feature3 = np.random.randn(n_observations)

In [31]:
feature0

array([False,  True,  True, ..., False,  True, False])

In [32]:
feature1

array([4, 4, 3, ..., 0, 4, 0])

In [33]:
# 您可以使用 _bytes_feature、_float_feature 或 _int64_feature 将下面的每个特征强制转换为兼容 tf.Example 的类型。然后，可以通过下面的已编码特征创建 tf.Example 消息：
def serialize_example(feature0, feature1, feature2, feature3):
  """
  Creates a tf.Example message ready to be written to a file.
  """
  # Create a dictionary mapping the feature name to the tf.Example-compatible
  # data type.
  feature = {
      'feature0': _int64_feature(feature0),
      'feature1': _int64_feature(feature1),
      'feature2': _bytes_feature(feature2),
      'feature3': _float_feature(feature3),
  }

  # Create a Features message using tf.train.Example.

  example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
  return example_proto.SerializeToString()

例如，假设您从数据集中获得了一个观测值 [False, 4, bytes('goat'), 0.9876]。您可以使用 create_message() 创建和打印此观测值的 tf.Example 消息。如上所述，每个观测值将被写为一条 Features 消息。请注意，tf.Example 消息只是 Features 消息外围的包装器：

In [36]:
example_observation = []

serialized_example = serialize_example(False, 4, b'goat', 0.9876)
serialized_example

b'\nR\n\x11\n\x08feature0\x12\x05\x1a\x03\n\x01\x00\n\x11\n\x08feature1\x12\x05\x1a\x03\n\x01\x04\n\x14\n\x08feature2\x12\x08\n\x06\n\x04goat\n\x14\n\x08feature3\x12\x08\x12\x06\n\x04[\xd3|?'

In [37]:
# 要解码消息，请使用 tf.train.Example.FromString 方法。
example_proto = tf.train.Example.FromString(serialized_example)
example_proto

features {
  feature {
    key: "feature0"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "feature1"
    value {
      int64_list {
        value: 4
      }
    }
  }
  feature {
    key: "feature2"
    value {
      bytes_list {
        value: "goat"
      }
    }
  }
  feature {
    key: "feature3"
    value {
      float_list {
        value: 0.9876000285148621
      }
    }
  }
}